In [ ]:
# load netcdf files from CIOPS-West (one per month)
# load netcdf files from observational data (one)

# CIOPS-West:

# provided by mdunphy, downloaded May 11, 2021
# format: one netcdf file per day for hourly temp and salin
# example file: ts_W_y2019m12d09.nc
# stored here: C:\Users\Greig\Sync\6. SSMSP Model\Model Greig\Data\29. Water Properties\CIOPSWest

# measurement / observation data: 

# 1/ Older IOS Moored Acoustic Dopller Current Profiler (ADCP) Profile Data 
# https://catalogue.cioospacific.ca/dataset/bodc-meds-cf-v52_ios_adcp_moorings
# https://data.cioospacific.ca/erddap/tabledap/IOS_ADCP_Moorings.html
# 1979 - 2019,  48.1 <= Lat >= 48.7 ,  -125.3 <= Lon >= -124.1  (Juan de Fuca)
# Does not include salinity
# IOS_CUR_Moorings_1979to2019.nc

# 2/ IOS CTD Profiles
# https://data.cioospacific.ca/erddap/tabledap/IOS_CTD_Profiles.html
# https://catalogue.cioospacific.ca/dataset/ios_ctd_profiles
# available back to 1965
# extracted: 1975 - present, 48.1 <= Lat >= 48.7 ,  -125.3 <= Lon >= -124.1  (Juan de Fuca entrance)
#

## TOC: <a class="anchor" id="top"></a>
* [1. IOS DATA](#section-1)
    * [1a. Moored Acoustic Doppler Current Profiler (ADCP)](#section-1a)
    * [1b. IOS CTD Profile](#section-1b) 
    * [1c. Combine IOS ADCP & CTD Data](#section-1c) 
* [2. CIOPS Data](#section-2) 

In [6]:
%matplotlib notebook
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc


In [24]:
########################### CIOPSw Data ##############################
# extract temp. and salin. for a coordinate from the ORAS5 files. 
# (one file per day)
start_year = 2016
max_year = 2019
CIOPSw_path = "C://Users//Greig//Sync//6. SSMSP Model//Model Greig//Data//29. Water Properties//CIOPSWest//bdy_ts//"
test_file = "ts_N_y2016m04d23.nc"

# Load CIOPS daily temp .nc
def load_CIOPS_temp(f, i, j):
    with nc.Dataset(f) as ncid:
        print(ncid.variables)
        time_raw = ncid.variables["time_counter"][:].filled()
        #depth = ncid.variables["deptht"][:].filled()
  
        ' temp dimensions: time, depth, lon, lat'
        temp = ncid.variables["votemper"][:,:,i,j].filled()
        
    return time_raw, temp

def numberOfDays(y, m):
    leap = 0
    if y% 400 == 0:
        leap = 1
    elif y % 100 == 0:
        leap = 0
    elif y% 4 == 0:
         leap = 1
    if m==2:
        return 28 + leap
    list = [1,3,5,7,8,10,12]
    if m in list:
        return 31
    return 30

time_raw, temp = load_CIOPS_temp(CIOPSw_path + test_file,2,2)

{'votemper': <class 'netCDF4._netCDF4.Variable'>
float32 votemper(time_counter, deptht, yb, xb)
    units: deg C
    grid: bdyT
unlimited dimensions: time_counter
current shape = (24, 40, 10, 10)
filling on, default _FillValue of 9.969209968386869e+36 used, 'vosaline': <class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, yb, xb)
    units: g/kg
    grid: bdyT
unlimited dimensions: time_counter
current shape = (24, 40, 10, 10)
filling on, default _FillValue of 9.969209968386869e+36 used, 'time_counter': <class 'netCDF4._netCDF4.Variable'>
float64 time_counter(time_counter)
    standard_name: time
    units: seconds since 1950-01-01 00:00:00
    calendar: gregorian
unlimited dimensions: time_counter
current shape = (24,)
filling on, default _FillValue of 9.969209968386869e+36 used}


In [26]:
temp.shape


(24, 40)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
viz_tools.set_aspect(ax, coords = 'map')

with nc.Dataset('..//data//bathymetry//bathy_salishsea_1500m_20210406.nc') as grid:
    viz_tools.plot_coastline(ax, grid, coords = 'map')

ax.plot(lont, latt, '.k')
ax.set_ylim(48, 49)
ax.legend()
ax.set_xlim(-125.5, -124)
plt.xticks(rotation=90)